#Note: I do not have enough computer power to train this. As per the last review i have added Bits and Bytes but i wont be Able to get results becuase of compute issues. Please approve on the basis of code only.

In [1]:
pip install transformers datasets peft bitsandbytes torch trl huggingface_hub

In [2]:
from huggingface_hub import login
login()

In [3]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import bitsandbytes as bnb

In [4]:
# 1. Load the dataset
large_dataset = load_dataset("FinGPT/fingpt-fiqa_qa", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
dataset = large_dataset.select(range(int(0.01 * len(large_dataset))))

print(f"Original dataset size: {len(large_dataset)}")
print(f"Reduced dataset size: {len(dataset)}")

Original dataset size: 17110
Reduced dataset size: 171


## Quantization

In [6]:
from transformers import BitsAndBytesConfig

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True
)

In [8]:
# 2. Load tokenizer and model
model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config = bnb_config
)
# 3. Set padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# 4. Format the dataset
def format_prompt(example):
    return {
        "text": f"""<|startofinstruction|> {example['instruction']}

Question: {example['input']}

Answer: {example['output']} <|endoftext|>"""
    }

# Process dataset
formatted_dataset = dataset.map(format_prompt)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## LoraConfig

In [9]:
# 5. Configure QLoRA
lora_config = LoraConfig(
    r=8,  # rank
    lora_alpha=32,  # scaling factor
    target_modules=["q_proj", "v_proj"],  # target attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [10]:
# Apply QLoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,597,440 || all params: 2,615,939,328 || trainable%: 0.0611


In [11]:

# 6. Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

# Tokenize dataset
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset.column_names
)

In [14]:
# 7. Training arguments
training_args = TrainingArguments(
    output_dir="./gemma-2-2b-it-fingpt-qlora",
    # per_device_train_batch_size=4,
    # gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=500,  # adjust based on your needs
    warmup_steps=50,
    fp16=True,
    bf16=False,
    report_to="tensorboard",
    push_to_hub=False,
)

# 8. Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    peft_config=lora_config,
    # dataset_text_field="text",
    # tokenizer=tokenizer,
    # max_seq_length=512,
)

# 9. Train the model
trainer.train()

# 10. Save the model
trainer.save_model("./gemma-2-2b-it-fingpt-qlora-final")
tokenizer.save_pretrained("./gemma-2-2b-it-fingpt-qlora-final")

# 11. Optional: Push to Hugging Face Hub
# trainer.push_to_hub("your-username/gemma-2-2b-it-fingpt-qlora")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 22.12 MiB is free. Process 84142 has 14.72 GiB memory in use. Of the allocated memory 14.32 GiB is allocated by PyTorch, and 283.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
          